In [2]:
import geopandas as gpd
import numpy as np
import pandana as pdna
import pandas as pd
from glob import glob
import matplotlib.pyplot as plt
from modules.Base import BaseModule

import warnings
warnings.filterwarnings("ignore")

In [3]:
class Streets(BaseModule):
    def __init__(self) -> None:
        super().__init__()
        self.nodes_cols = []
        self.edges_cols = []
        self.scenarios_status = [0]*len(self.plates)
        pass

    def load_data(self):
        self.scenarios = []
        [
            self.scenarios.append({
                    # 'nodes': gpd.read_file(node_file).to_crs(self.default_crs),
                    # 'edges': gpd.read_file(edge_file).to_crs(self.default_crs)
                    'nodes': gpd.read_parquet(node_file),
                    'edges': gpd.read_parquet(edge_file)
            })
            for node_file, edge_file in zip(
                glob('/app/assets/streets/*/nodes.parquet'),
                glob('/app/assets/streets/*/edges.parquet')
                )
        ]
        # Assume that the baseline is in index 0
        self.current_scenario = self.scenarios[0]
        self.make_network()
        pass

    def update_network(self, plate_id, scenario_id):
        self.update_plate_area(plate_id=plate_id, scenario_id=scenario_id)
        self.make_network()
        pass

    def update_plate_area(self, plate_id, scenario_id):
        # Change the nodes and edges within the area into plate

        current_nodes = self.current_scenario['nodes']
        current_edges = self.current_scenario['edges']

        update_nodes = self.scenarios[scenario_id]['nodes']
        update_edges = self.scenarios[scenario_id]['edges']

        mask_current_nodes = current_nodes['plate_id'] != plate_id
        mask_current_edges = current_edges['plate_id'] != plate_id

        mask_update_nodes = update_nodes['plate_id'] == plate_id
        mask_update_edges = update_edges['plate_id'] == plate_id
        
        edges_updated = pd.concat([
            current_edges[mask_current_edges],
            update_edges[mask_update_edges]
            ]).drop_duplicates(subset=['u', 'v', 'key'])
        
        # Fix missing nodes
        look_nodes = list(edges_updated['u'].values) + list(edges_updated['v'].values)
        
        fixed_current_nodes = current_nodes[
            current_nodes['osmid'].isin(look_nodes)
        ]

        fixed_update_nodes = update_nodes[
            update_nodes['osmid'].isin(look_nodes)
        ]

        # Clean duplicates
        nodes_updated = pd.concat([
            current_nodes[mask_current_nodes],
            update_nodes[mask_update_nodes],
            fixed_current_nodes,
            fixed_update_nodes,
        ]).drop_duplicates(subset=['osmid'])
        
        # Only nodes with edges connected
        nodes_updated = nodes_updated[
            nodes_updated['osmid'].isin(edges_updated['u']) | nodes_updated['osmid'].isin(edges_updated['v'])
        ]

        self.current_scenario = {
            'nodes': gpd.GeoDataFrame(data=nodes_updated.drop(columns=['geometry']), geometry=nodes_updated['geometry']),
            'edges': gpd.GeoDataFrame(data=edges_updated.drop(columns=['geometry']), geometry=edges_updated['geometry']),
        }
        pass

    def current_nodes_edges_to_net_format(self):
        nodes_df = self.current_scenario['nodes']
        edges_df = self.current_scenario['edges']

        nodes = pd.DataFrame(
            {
                'osmid': nodes_df['osmid'].astype(int),
                'lat' : nodes_df.geometry.y.astype(float),
                'lon' : nodes_df.geometry.x.astype(float),
                'y' : nodes_df.geometry.y.astype(float),
                'x' : nodes_df.geometry.x.astype(float),
            }
        )
        nodes['id'] = nodes['osmid'].values

        nodes = gpd.GeoDataFrame(data=nodes, geometry=nodes_df.geometry)
        nodes.set_index('osmid', inplace=True)
        nodes.drop_duplicates(inplace=True)

        edges = pd.DataFrame(
            {
                'u': edges_df['u'].astype(int),
                'v': edges_df['v'].astype(int),
                'from': edges_df['u'].astype(int),
                'to': edges_df['v'].astype(int),
                'osmid': edges_df['osmid'].astype(int),
                'length': edges_df['length'].astype(float)
            }
        )
        edges['key'] = 0
        edges['key'] = edges['key'].astype(int)
        edges = gpd.GeoDataFrame(data=edges, geometry=edges_df.geometry)
        edges.set_index(['u', 'v', 'key'], inplace=True)
        edges.drop_duplicates(inplace=True)
        return nodes, edges
    
    def make_network(self):
        nodes, edges = self.current_nodes_edges_to_net_format()
        self.current_scenario['net'] = pdna.Network(
            nodes['lon'],
            nodes['lat'],
            edges['from'],
            edges['to'],
            edges[['length']]
        )
        pass

    def get_current_nodes_and_edges(self):
        return self.current_scenario['nodes'], self.current_scenario['edges']

In [4]:
st = Streets()
st.load_data()
# plate_id = 6
# st.update_network(plate_id=plate_id, scenario_id=1)

Generating contraction hierarchies with 16 threads.
Setting CH node vector of size 35909
Setting CH edge vector of size 79532
Range graph removed 79532 edges of 159064
. 10% . 20% . 30% . 40% . 50% . 60% . 70% . 80% . 90% . 100%


In [7]:
st.plates

{'1':    Id                                           geometry
 0   1  POLYGON ((672512.460 5922774.884, 672641.685 5...,
 '2':    Id                                           geometry
 0   2  POLYGON ((672544.141 5922582.191, 672571.155 5...,
 '3':    Id                                           geometry
 0   3  POLYGON ((672457.570 5922128.930, 672555.655 5...,
 '4':    Id                                           geometry
 0   4  POLYGON ((672674.866 5921866.639, 672592.588 5...,
 '5':    Id                                           geometry
 0   5  POLYGON ((673024.404 5921889.716, 673174.226 5...,
 '6':    Id                                           geometry
 0   6  POLYGON ((672865.890 5921410.949, 673117.144 5...,
 '7':    Id                                           geometry
 0   7  POLYGON ((673027.298 5921083.472, 673240.231 5...,
 '8':    Id                                           geometry
 0   8  POLYGON ((673193.551 5920789.462, 673410.984 5...,
 'all':    Id           